# 将输出部件与日志模块集成

虽然使用 `.capture` 装饰器对于理解和调试单个回调很有用，但它不适用于更大的应用程序。通常，在较大的应用程序中，人们可能会使用 [logging](https://docs.python.org/3/library/logging.html) 模块来打印程序状态的信息。然而，在部件应用程序的情况下，不清楚日志输出应该去哪里。

一个有用的模式是创建一个自定义的 [handler](https://docs.python.org/3/library/logging.html#handler-objects)，它将日志重定向到输出部件。然后可以在新的单元格中显示输出部件，以在应用程序运行时监视它。

In [1]:
# 为了保证 `JupyterLite` 可用，需要 notebook 开头添加：
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%', 
            'height': '160px', 
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout', 
            'output_type': 'stream', 
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs
        
    def show_logs(self):
        """ Show the logs """
        display(self.out)
    
    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [3]:
handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [4]:
handler.clear_logs()
logger.info('Starting program')

try:
    logger.info('About to try something dangerous...')
    1.0/0.0
except Exception as e:
    logger.exception('An error occurred!')